<a href="https://colab.research.google.com/github/JulianMeigen/ML-handson/blob/main/notebooks/7.0-SNJMMH-Day7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment Day 7

## Team members:
- Samuel Nebgen s6sanebg@uni-bonn.de
- Muhammad Humza Arain s27marai@uni-bonn.de
- Julian Meigen s82jmeig@uni-bonn.de

## 16.09.2025

Contributions were made by all team members in around the same amount, either based on discussions or coding.

In [24]:
!gdown --folder https://drive.google.com/drive/folders/1VESm-JaHEqPJmM23iLW1mEJsuI2mLBdx?usp=sharing

Retrieving folder contents
Processing file 1i6W9fI3sGEn6V9xBlt2MxlonZXOTLZjg load-subgraph_doc.ipynb
Processing file 1qZpQzFMRzuYQ0xoQJcUNe7CRBMS2mRmz subgraph_hop_1.pt
Processing file 1iz_FOBs9k7m9z3lDtRIXzRkd92tL_EK- subgraph.pt
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1i6W9fI3sGEn6V9xBlt2MxlonZXOTLZjg
To: /content/ML-HandsOn/load-subgraph_doc.ipynb
100% 2.92k/2.92k [00:00<00:00, 15.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1qZpQzFMRzuYQ0xoQJcUNe7CRBMS2mRmz
To: /content/ML-HandsOn/subgraph_hop_1.pt
100% 10.5M/10.5M [00:00<00:00, 59.3MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1iz_FOBs9k7m9z3lDtRIXzRkd92tL_EK-
From (redirected): https://drive.google.com/uc?id=1iz_FOBs9k7m9z3lDtRIXzRkd92tL_EK-&confirm=t&uuid=969957c7-9308-4ed3-a29c-57dd17fd67e5
To: /content/ML-HandsOn/subgraph.pt
100% 1.64G/1.64G [00:26<00:00, 62.7MB/s]
Download c

In [25]:
!pip install torch_geometric

In [26]:
import torch
import torch_geometric
import numpy as np
import pandas as pd
import networkx as nx
import plotly
from torch_geometric.utils import to_networkx
from torch.nn import Embedding
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

# Task 1 Perform a node labeling task with a Graph ML model

## a) Load the graph dataset (ogbn-proteins) into pytorch-geometric

We are directly using a Subgraph

In [27]:
import torch
import torch_geometric

path_big = "/content/ML-HandsOn/subgraph.pt"
path_small = "/content/ML-HandsOn/subgraph_hop_1.pt"

dataset = torch.load(path_small, weights_only=False)

data = dataset["graph"]

print(data)

Data(num_nodes=942, edge_index=[2, 200414], edge_attr=[200414, 8], node_species=[942, 1], y=[942, 112])


In [28]:
G = to_networkx(data, to_undirected=True)
print(G)

Graph with 942 nodes and 100207 edges


## b) Create a train, val, test split on the nodes or load the masks via pytorch-geometric.

### i. Create a subgraph if the computation is too expensive.

In [54]:
num_nodes = data.num_nodes
perm = torch.randperm(num_nodes)

train_size = int(0.7 * num_nodes)
val_size = int(0.15 * num_nodes)

train_mask = torch.zeros(num_nodes, dtype=torch.bool)
val_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)

train_mask[perm[:train_size]] = True
val_mask[perm[train_size:train_size + val_size]] = True
test_mask[perm[train_size + val_size:]] = True

data.train_mask = train_mask
data.val_mask = val_mask
data.test_mask = test_mask

print(data.y)

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])


In [55]:
print(len(data.y[data.train_mask]))
print(len(data.y[data.val_mask]))
print(len(data.y[data.test_mask]))

659
141
142


## c) Initialize the graph with random node embeddings.

In [56]:
number_nodes = data.num_nodes
embedding_dim = 64
x = torch.empty((num_nodes, embedding_dim))  # empty tensor
torch.nn.init.xavier_uniform_(x)  # Xavier uniform initialization
data.x = x

In [82]:
node_idx = np.arange(data.num_nodes)
train_idx = torch.tensor(node_idx[data.train_mask.numpy()])
test_idx = torch.tensor(node_idx[data.test_mask.numpy()])
val_idx = torch.tensor(node_idx[data.val_mask.numpy()])

train_subgraph = data.subgraph(train_idx)
test_subgraph = data.subgraph(test_idx)
val_subgraph = data.subgraph(val_idx)

## d) Define a graph convolutional neural network class with two layers using pytorch-geometric..

In [41]:
class GCN(torch.nn.Module):
    def __init__(self, num_nodes, embedding_dim, hidden_dim, out_dim):
        super().__init__()
        self.conv1 = GCNConv(embedding_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, out_dim)
        self.dropout = torch.nn.Dropout(0.5)

    def forward(self, x, edge_index):
        # apply GCN layers
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [42]:
model_gcn = GCN(num_nodes=data.num_nodes, embedding_dim=64, hidden_dim=128, out_dim=112)

### i. Train your model on the train dataset using an optimizer and a loss function for a multilabel classification task for 100 epochs

tensor([[  0, 676,   0,  ..., 755, 937, 938],
        [676,   0, 937,  ..., 804, 938, 937]])

In [53]:
train_subgrap = data.subgraph()

torch.Size([942])


Data(num_nodes=942, edge_index=[2, 200414], edge_attr=[200414, 8], node_species=[942, 1], y=[942, 112], train_mask=[942], val_mask=[942], test_mask=[942], x=[942, 64])

In [83]:
# Optimizer and loss
optimizer = torch.optim.Adam(model_gcn.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.BCEWithLogitsLoss()

# Training loop
epochs = 100
model_gcn.train()
for epoch in range(1, epochs + 1):
    optimizer.zero_grad()
    out = model_gcn(data.x, train_subgraph.edge_index)  # forward pass
    loss = criterion(out, data.y.float())        # multi-label BCE loss
    loss.backward()                       # backward pass
    optimizer.step()                      # update parameters

    if epoch % 10 == 0 or epoch == 1:
        print(f"Epoch {epoch:03d}, Loss: {loss.item():.4f}")

Epoch 001, Loss: 1.2703
Epoch 010, Loss: 1.2705
Epoch 020, Loss: 1.2695
Epoch 030, Loss: 1.2693
Epoch 040, Loss: 1.2693
Epoch 050, Loss: 1.2693
Epoch 060, Loss: 1.2693
Epoch 070, Loss: 1.2693
Epoch 080, Loss: 1.2693
Epoch 090, Loss: 1.2690
Epoch 100, Loss: 1.2690


### ii. Test your model on the test set and evaluate it with accuracy, AUROC, precision, recall and F1 score.

In [ ]:
model_gcn.eval()
with torch.no_grad():
    out = model_gcn(data.x, data.edge_index)
    predicted_labels = torch.sigmoid(out) > 0.5

## e) Set up a hyperparameter optimization pipeline with nested 5-fold cross-validation

### i. Familiarize yourself with the hyperparameter optimization package optuna (https://optuna.org/ )

### ii. Integrate the logging package mlflow (https://mlflow.org/) to log your metrics.

### iii. Train and test your models and report the evaluation metrics with mean and std for the nested CV.